# NRC-23 - Image Quality Verification by Filter   

## Notebook: Comparison Color-Magnitude Diagram - PSF photometry

**Author**: Matteo Correnti, STScI Scientist II
<br>
**Created**: October, 2021
<br>
**Last Updated**: February, 2022

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
    2.2 [Plotting functions imports](#matpl_imports)<br>
3. [Import images to analyze](#data)<br>
    3.1 [Select Detector/Filter to analyze](#sel_data)<br>
    3.2 [Transform the images to data models](#transf_images)<br>
    3.3 [Load the PSF photometry catalogs](#load_cat)<br>
    3.4 [Clean the catalogs](#clean_cat)<br>
4. [Transform pixel to equatorial coordinates](#transf_coord)<br>
5. [Cross-Match the catalogs](#cross-match)<br>
6. [Comparison with LMC astrometric catalog](#comp_lmc)<br>
    6.1 [Import LMC astrometric catalog](#load_lmc_cat)<br>
    6,2 [Select a sub-region coincident with our field of view](#fov_lmc)<br>
    6.3 [Cross-match with LMC astrometric catalog](#crossmatch_lmc)<br>
7. [Color-Magnitude Diagrams comparison](#cmd)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

This notebook shows a quick comparison between the Color-Magnitude (CMD) of the original LMC catalog (with NIRCam simulated magnitudes) and the instrumental (i.e., not-calibrated) CMD obtained from the PSF photometry.

**Dependencies**:  before running this notebook it is necessary to perform PSF photometry on the images, to obtain the source instrumental magnitudes, using the notebook `NRC-23_psf_photometry.ipynb`.

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all the necessary Python packages and we define some plotting parameters.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import os 

import sys
import time
import copy

import glob as glob

import numpy as np

import pandas as pd

from astropy import units as u
from astropy.io import fits
from astropy.table import Table, QTable
from astropy.stats import sigma_clipped_stats

import jwst
from jwst.datamodels import ImageModel

from astropy.coordinates import SkyCoord, match_coordinates_sky

import pysiaf

### 2.2<font color='white'>-</font>Plotting function imports<a class="anchor" id="matpl_imports"></a> ###

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 20

font1 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '12'}
font2 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '20'}

In [ ]:
figures_dir = 'FIGURES/'

if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

3.<font color='white'>-</font>Import images to analyze<a class="anchor" id="data"></a>
------------------

In [ ]:
dict_images = {'NRCA1': {}, 'NRCA2': {}, 'NRCA3': {}, 'NRCA4': {}, 'NRCA5': {},
               'NRCB1': {}, 'NRCB2': {}, 'NRCB3': {}, 'NRCB4': {}, 'NRCB5': {}}

dict_filter_short = {}
dict_filter_long = {}

ff_short = []
det_short = []
det_long = []
ff_long = []
detlist_short = []
detlist_long = []
filtlist_short = []
filtlist_long = []

images_dir = '../Simulation/Pipeline_Outputs/Level2_Outputs'
images = sorted(glob.glob(os.path.join(images_dir, "*cal.fits")))

for image in images:

    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p
    
    wv = float(f[1:3])

    if wv > 24:         
        ff_long.append(f)
        det_long.append(d)

    else:
        ff_short.append(f)
        det_short.append(d)   

    detlist_short = sorted(list(dict.fromkeys(det_short)))
    detlist_long = sorted(list(dict.fromkeys(det_long)))

    unique_list_filters_short = []
    unique_list_filters_long = []

    for x in ff_short:

        if x not in unique_list_filters_short:

            dict_filter_short.setdefault(x, {})
                 
    for x in ff_long:
        if x not in unique_list_filters_long:
            dict_filter_long.setdefault(x, {})   
            
    for d_s in detlist_short:
        dict_images[d_s] = copy.deepcopy(dict_filter_short)

    for d_l in detlist_long:
        dict_images[d_l] = copy.deepcopy(dict_filter_long)

    filtlist_short = sorted(list(dict.fromkeys(dict_filter_short)))
    filtlist_long = sorted(list(dict.fromkeys(dict_filter_long)))

print("Available Detectors for SW channel:", detlist_short)
print("Available Detectors for LW channel:", detlist_long)
print("Available SW Filters:", filtlist_short)
print("Available LW Filters:", filtlist_long)

In [ ]:
for image in images:
    
    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p

    if len(dict_images[d][f]) == 0:
        dict_images[d][f] = {'images': [image]}
    else:
        dict_images[d][f]['images'].append(image)


### 3.1<font color='white'>-</font>Select detector/filter to analyze<a class="anchor" id="sel_data"></a> ###

In [ ]:
det = 'NRCB1'
filt1 = 'F115W'
filt2 = 'F200W'

### 3.2<font color='white'>-</font>Transform the images to data models<a class="anchor" id="transf_images"></a> ###

In order to assign the WCS coordinate and hence cross-match the images, we need to transform the images to DataModel. The coordinates are assigned during the step [assign_wcs](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/main.html?#using-the-wcs-interactively) step in the JWST pipeline and allow us to cross-match the different catalogs obtained for each filter.

In [ ]:
images_1 = []
images_2 = []

for i in np.arange(0, len(dict_images[det][filt1]['images']), 1):

    image_1 = ImageModel(dict_images[det][filt1]['images'][i])
    images_1.append(image_1)
        
for i in np.arange(0, len(dict_images[det][filt2]['images']), 1):

    image_2 = ImageModel(dict_images[det][filt2]['images'][i])
    images_2.append(image_2)

### 3.3<font color='white'>-</font>Load the PSF photometry catalogs<a class="anchor" id="load_cat"></a> ###

In [ ]:
# adopted reduction parameters (needed to retrieve the photometry catalogs from the right folder)

npsfs = 16
th = 10
fitshape = (11, 11)

cat_dir = 'PSF_PHOT_OUTPUT/numPSFs{}_Th{}_fitshape{}x{}'.format(npsfs, th, fitshape[0], fitshape[1])
phots_pkl_1 = sorted(glob.glob(os.path.join(cat_dir, 'phot_'+str.lower(det)+'_'+filt1+'*.pkl')))
phots_pkl_2 = sorted(glob.glob(os.path.join(cat_dir, 'phot_'+str.lower(det)+'_'+filt2+'*.pkl')))

phots_pkl_1

results_1 = []
results_2 = []

for phot_pkl_1 in phots_pkl_1:
    ph_1 = pd.read_pickle(phot_pkl_1)
    result_1 = QTable.from_pandas(ph_1)
    results_1.append(result_1)

for phot_pkl_2 in phots_pkl_2:
    ph_2 = pd.read_pickle(phot_pkl_2)
    result_2 = QTable.from_pandas(ph_2)
    results_2.append(result_2)

### 3.4<font color='white'>-</font>Clean the catalogs<a class="anchor" id="clean_cat"></a> ###

In [ ]:
results_clean_1 = []
results_clean_2 = []

for i in np.arange(0, len(images_1), 1):

    mask_1 = ((results_1[i]['x_fit'] > 0) & (results_1[i]['x_fit'] < images_1[0].shape[1]) &
                  (results_1[i]['y_fit'] > 0) & (results_1[i]['y_fit'] < images_1[0].shape[0]) &
                  (results_1[i]['flux_fit'] > 0))

    result_clean_1 = results_1[i][mask_1]
    result_clean_1[filt1+'_inst'] = -2.5 * np.log10(result_clean_1['flux_fit'])
    result_clean_1['e'+filt1+'_inst'] = 1.086 * (result_clean_1['flux_unc'] / result_clean_1['flux_fit'])

    results_clean_1.append(result_clean_1)

for i in np.arange(0, len(images_2), 1):
    
    mask_2 = ((results_2[i]['x_fit'] > 0) & (results_2[i]['x_fit'] < images_2[0].shape[1]) &
                  (results_2[i]['y_fit'] > 0) & (results_2[i]['y_fit'] < images_2[0].shape[0]) &
                  (results_2[i]['flux_fit'] > 0))

    result_clean_2 = results_2[i][mask_2]
    result_clean_2[filt2+'_inst'] = -2.5 * np.log10(result_clean_2['flux_fit'])
    result_clean_2['e'+filt2+'_inst'] = 1.086 * (result_clean_2['flux_unc'] / result_clean_2['flux_fit'])

    results_clean_2.append(result_clean_2)

4.<font color='white'>-</font>Transform pixel to equatorial coordinates<a class="anchor" id="transf_coord"></a>
------------------

We can transform the (x,y) positions from the raw, NIRCam coordinate system to the Equatorial system (ICRS) by means of the WCS information stored in the ASDF extension of our image.

<div class="alert alert-block alert-info">
    <h3><u><b>NB</b></u></h3>
    
For stage-2 images (_cal.fits_ files), the WCS information is assigned by the _assign_wcs_ step. This information is saved in an ASDF extension of the FITS file. However, image-display tools such as ds9 do not understand the ASDF encoding. For this reason, an approximation to the WCS is stored in the image header using the SIP (Simple Imaging Polynomial) convention. The SIP-based header does not provide an exact fit: it is accurate to a $\sim$0.25-pixel level and it is meant for general display purposes (see <a href="https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html">here</a> for more information).
</ul>
</div>

In [ ]:
results_final_1 = []
results_final_2 = []

for i in np.arange(0, len(images_1), 1):
    
    result_final_1 = results_clean_1[i]
    ra_1, dec_1 = images_1[i].meta.wcs(result_final_1['x_fit'], result_final_1['y_fit'])
    radec_1 = SkyCoord(ra_1, dec_1, unit='deg')
    result_final_1['radec'] = radec_1
    results_final_1.append(result_final_1)


for i in np.arange(0, len(images_2), 1):

    result_final_2 = results_clean_2[i]
    ra_2, dec_2 = images_2[i].meta.wcs(result_final_2['x_fit'], result_final_2['y_fit'])
    radec_2 = SkyCoord(ra_2, dec_2, unit='deg')
    result_final_2['radec'] = radec_2
    results_final_2.append(result_final_2)

5.<font color='white'>-</font>Cross-match the catalogs<a class="anchor" id="cross-match"></a>
------------------

We cross-match the catalogs to obtain the difference in positions between the different filters.

Stars from the two filters are associated if the distance between the matches is < 0.5 px.   

We can cross-match each single catalog for the two filters:

In [ ]:
max_sep = 0.5
pixelscale = 0.031 # select the right pixelscale for SW or LW detector

matches_phot_1 = []
matches_phot_2 = []

for res1 in results_final_1:
    
    for res2 in results_final_2:
    
        match_phot_1 = Table()
        match_phot_2 = Table()
    
        idx, d2d, _ = match_coordinates_sky(res1['radec'], res2['radec'])
        sep_constraint = (d2d.arcsec / pixelscale) < max_sep
    
        match_phot_1 = res1[sep_constraint]
        match_phot_2 = res2[idx[sep_constraint]]
    
        matches_phot_1.append(match_phot_1)
        matches_phot_2.append(match_phot_2)
    

6.<font color='white'>-</font>Comparison with LMC astrometric catalog<a class="anchor" id="comp_lmc"></a>
------------------

### 6.1<font color='white'>-</font>Import LMC astrometric catalog<a class="anchor" id="load_lmc_cat"></a> ###

In [ ]:
input_cat = '../Simulation/lmc_pointsources.cat'

names = ['index', 'ra_in', 'dec_in', 'F070W', 'F090W', 'F115W', 'F140M', 'F150W2', 'F150W', 'F162M', 'F164N', 
         'F182M', 'F187N', 'F200W', 'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2', 'F323N', 'F335M', 
         'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W', 'F460M', 'F466N', 'F470N', 'F480M']

cat = pd.read_csv(input_cat, header=None, sep='\s+', names=names,
                  comment='#', skiprows=7, usecols=(0,1,2,5,13))
cat.head()

### 6.2<font color='white'>-</font>Select a sub-region coincident with our field of view<a class="anchor" id="fov_lmc"></a> ###

In [ ]:
print('Total number of stars in the LMC catalog:', len(cat))

xlim0 = 0
ylim0 = 0
xlim1 = images_1[0].shape[1]
ylim1 = images_1[0].shape[0]

ra_lim0, dec_lim0 = images_1[0].meta.wcs(xlim0, ylim0)
print(ra_lim0, dec_lim0)
ra_lim1, dec_lim1 = images_1[0].meta.wcs(xlim1 - 1, ylim1 - 1)
print(ra_lim1, dec_lim1)

ra_lim = sorted([ra_lim0, ra_lim1])
dec_lim = [dec_lim0, dec_lim1]

cat_sel = cat.loc[(cat['ra_in'] > ra_lim[0]) & (cat['ra_in'] < ra_lim[1]) & (cat['dec_in'] > dec_lim[0]) & 
              (cat['dec_in'] < dec_lim[1])]

print('Number of LMC stars in the field of view:', len(cat_sel))
#cat_sel.head()

### 6.3<font color='white'>-</font>Cross-match with LMC astrometric catalog<a class="anchor" id="crossmatch_lmc"></a> ###

In [ ]:
radec_input = SkyCoord(cat_sel['ra_in'], cat_sel['dec_in'], unit='deg')
catalog1 = matches_phot_1[0]
catalog2 = matches_phot_2[0]

idx_inp, d2d_inp, _ = match_coordinates_sky(radec_input, catalog1['radec'])
sep_constraint_inp = (d2d_inp.arcsec / pixelscale) < max_sep
match_phot_inp = Table()
match_phot_inp['radec'] = radec_input[sep_constraint_inp]
match_phot_inp[filt1] = cat_sel[filt1][sep_constraint_inp]
match_phot_inp[filt2] = cat_sel[filt2][sep_constraint_inp]
match_phot_out_1 = catalog1[idx_inp[sep_constraint_inp]]
match_phot_out_2 = catalog2[idx_inp[sep_constraint_inp]]
print('Number of sources cross-matched with the LMC catalog:', len(match_phot_out_1))

7.<font color='white'>-</font>Color-Magnitude Diagrams comparison<a class="anchor" id="cmd"></a>
------------------

In [ ]:
plt.figure(figsize=(14, 12))

ax1 = plt.subplot(1, 2, 1)

ax1.set_xlabel(filt1+ ' - '+filt2, fontdict=font2)
ax1.set_ylabel(filt1, fontdict=font2)

ax1.set_title('LMC Catalog Input', fontdict=font2)

plt.suptitle(det, fontsize=30)

col_inp = cat_sel[filt1] - cat_sel[filt2]
mag_inp = cat_sel[filt1]

xlim0 = -1
xlim1 = 1.5
ylim0 = 24
ylim1 = 12

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.scatter(col_inp, mag_inp, c='k', s=10)

col_inp_match =  match_phot_inp[filt1] - match_phot_inp[filt2]
mag_inp_match = match_phot_inp[filt1]

ax1.scatter(col_inp_match, mag_inp_match, c='r', s=20)
ax1.text(xlim0+0.1, ylim1+0.5, 'LMC sources FOV: {}'.format(len(mag_inp)), fontdict=font2)
ax1.text(xlim0+0.1, ylim1+1.0, 'Cross-matched with PSF phot: {}'.format(len(mag_inp_match)) , color='r', 
        fontdict=font2)

ax2 = plt.subplot(1, 2, 2)

ax2.set_xlabel(filt1+'_inst - '+filt2+'_inst', fontdict=font2)
ax2.set_ylabel(filt1+'_inst', fontdict=font2)

ax2.set_title('PSF photometry', fontdict=font2)

col_out = catalog1[filt1+'_inst'] - catalog2[filt2+'_inst']
mag_out = catalog1[filt1+'_inst']

xlim0 = -1.5
xlim1 = 1
ylim0 = -1
ylim1 = -11

ax2.set_xlim(xlim0, xlim1)
ax2.set_ylim(ylim0, ylim1)

ax2.xaxis.set_major_locator(ticker.AutoLocator())
ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax2.yaxis.set_major_locator(ticker.AutoLocator())
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax2.scatter(col_out, mag_out, c='k', s=10)

col_out_match = match_phot_out_1[filt1+'_inst'] - match_phot_out_2[filt2+'_inst']
mag_out_match = match_phot_out_1[filt1+'_inst']

#ax2.scatter(col_out_match, mag_out_match, c='r', s=5)

plt.tight_layout()

filename = 'CMD_psf_phot_{0}_{1}-{2}vs{1}.png'.format(det, filt1, filt2)

#plt.savefig(os.path.join(figures_dir, filename))